In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
!pip install sweetviz
import sweetviz as sv
import os

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
notebook_dir = os.getcwd() 
path = notebook_dir + "\\Data"

# Construimos la ruta al archivo SQLite
sqlite_path = os.path.join(path, 'NBA_DATABASE.sqlite')

# Conectamos SQLite usando la ruta
conn = sqlite3.connect(sqlite_path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

tables = [table[0] for table in tables]

for table_name in tables:
    print(f"Table: {table_name}")


Table: game_info
Table: team_stats
Table: player_stats
Table: player_full_10
Table: player_full_20
Table: player_full_30


In [ ]:
# query = """ALTER TABLE player_stats
# ADD COLUMN date DATE;
# """

# cursor.execute(query)
# conn.commit()

# query2 = """UPDATE player_stats
# SET date = (SELECT game_info.date FROM game_info WHERE game_info.game_id = player_stats.game_id)
# WHERE EXISTS (SELECT 1 FROM game_info WHERE game_info.game_id = player_stats.game_id);
# """

# cursor.execute(query2)
# conn.commit()
# player_stats = pd.read_sql_query("SELECT * from player_stats", conn)
# player_stats.head()

,game_id,player,team,MP,FG,FGA,FG%,3P,3PA,3P%,...,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,PIE,date
0,192010220001,Jrue Holiday,NOP,41:05,6.0,15.0,0.400,1.0,6.0,0.167,...,22.0,0.0,4.1,23.9,20.8,81.0,120.0,-6.9,2.6,2019-10-22
1,192010220001,Brandon Ingram,NOP,35:06,8.0,19.0,0.421,2.0,5.0,0.400,...,24.4,1.3,4.8,8.8,26.5,111.0,113.0,3.5,8.8,2019-10-22
2,192010220001,JJ Redick,NOP,27:03,6.0,9.0,0.667,4.0,6.0,0.667,...,6.3,0.0,0.0,25.0,18.1,112.0,121.0,1.0,5.2,2019-10-22
3,192010220001,Lonzo Ball,NOP,24:50,2.0,7.0,0.286,2.0,3.0,0.667,...,27.6,0.0,0.0,11.3,14.6,119.0,116.0,-0.9,5.2,2019-10-22
4,192010220001,Derrick Favors,NOP,20:46,3.0,6.0,0.500,0.0,0.0,NaN,...,14.4,0.0,4.1,14.3,13.8,104.0,110.0,-3.4,3.1,2019-10-22


In [8]:
game_info = pd.read_sql_query("SELECT * from game_info", conn)
game_info[game_info['season']== 910]

,game_id,season,date,away_team,away_score,home_team,home_score,result
5830,91010270001,910,2009-10-27,BOS,95,CLE,89,0
5831,91010270002,910,2009-10-27,WAS,102,DAL,91,0
5832,91010270003,910,2009-10-27,HOU,87,POR,96,1
5833,91010270004,910,2009-10-27,LAC,92,LAL,99,1
5834,91010280005,910,2009-10-28,IND,109,ATL,120,1
...,...,...,...,...,...,...,...,...
7055,91004141226,910,2010-04-14,NYK,113,TOR,131,1
7056,91004141227,910,2010-04-14,IND,97,WAS,98,1
7057,91004141228,910,2010-04-14,LAL,91,LAC,107,1
7058,91004141229,910,2010-04-14,GSW,122,POR,116,0


In [9]:
team_stats = pd.read_sql_query("SELECT * from team_stats", conn)
team_stats.head()

,game_id,team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,192010220001,NOP,265,43.0,102.0,0.422,19.0,45.0,0.422,17.0,...,69.8,48.2,69.8,3.5,14.3,14.6,100.0,107.6,114.7,None
1,192010220001,TOR,265,42.0,103.0,0.408,14.0,40.0,0.350,32.0,...,71.9,51.8,54.8,6.2,5.3,12.4,100.0,114.7,107.6,None
2,192010220002,LAL,240,37.0,85.0,0.435,13.0,33.0,0.394,15.0,...,74.4,47.7,54.1,4.2,14.0,13.7,100.0,106.5,116.9,None
3,192010220002,LAC,240,42.0,81.0,0.519,11.0,31.0,0.355,17.0,...,79.1,52.3,57.1,8.4,9.6,13.3,100.0,116.9,106.5,None
4,192010230003,CHI,240,49.0,105.0,0.467,9.0,30.0,0.300,18.0,...,76.3,54.4,51.0,10.7,9.1,9.5,100.0,121.8,122.8,None


In [10]:
player_stats = pd.read_sql_query("SELECT * from player_stats", conn)
player_stats.columns

Index(['game_id', 'player', 'team', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'BPM', 'PIE', 'date'],
      dtype='object')

In [ ]:
# Reordenamos para tener date más a la vista
columnas = list(player_stats.columns)
columnas.insert(1, columnas.pop(columnas.index('date')))
player_stats = player_stats[columnas]
player_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455813 entries, 0 to 455812
Data columns (total 41 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   game_id  455813 non-null  int64  
 1   date     455813 non-null  object 
 2   player   455813 non-null  object 
 3   team     455813 non-null  object 
 4   MP       455813 non-null  object 
 5   FG       377829 non-null  float64
 6   FGA      377829 non-null  float64
 7   FG%      359665 non-null  float64
 8   3P       377829 non-null  float64
 9   3PA      377829 non-null  float64
 10  3P%      258226 non-null  float64
 11  FT       377829 non-null  float64
 12  FTA      377829 non-null  float64
 13  FT%      215804 non-null  float64
 14  ORB      377829 non-null  float64
 15  DRB      377829 non-null  float64
 16  TRB      377829 non-null  float64
 17  AST      377829 non-null  float64
 18  STL      377829 non-null  float64
 19  BLK      377829 non-null  float64
 20  TOV      377829 non-null  

In [ ]:
# Función para obtener los últimos números de partidos jugados por el roster de un equipo en un partido

def roster_query (game_id, n_games, team="away"):

  stats_columns = ", ".join([f"ps.`{col}`" for col in player_stats.columns if col not in ['game_id', 'player', 'team', 'date']])
  stats_columns2 = ", ".join([f"rg.`{col}`" for col in player_stats.columns if col not in ['game_id', 'player', 'team', 'date']])
  stats_columns3 = ", ".join([f"fg.`{col}`" for col in player_stats.columns if col not in ['game_id', 'player', 'team', 'date']])
  season = game_info[game_info['game_id'] == game_id]['season'].values[0]

  if team == "away":
    roster_query = f'''
    WITH CurrentGame AS (
        SELECT
            gi.away_team,
            gi.game_id AS current_game_id,
            gi.date AS current_game_date
        FROM
            game_info gi
        WHERE
            gi.game_id = {game_id}
    ),
    AwayTeamPlayers AS (
        SELECT
            ps.player,
            ps.game_id
        FROM
            player_stats ps
        JOIN
            game_info gi
            ON ps.game_id = gi.game_id
        JOIN
            CurrentGame cg
            ON gi.game_id = cg.current_game_id
        WHERE
            ps.team = cg.away_team -- Solo jugadores del equipo visitante
    ),
    LastNGames AS (
        SELECT
            gi.date
        FROM
            game_info gi
        JOIN
            CurrentGame cg
            ON (gi.away_team = cg.away_team OR gi.home_team = cg.away_team)
        WHERE
            gi.date < cg.current_game_date -- Solo fechas anteriores
        ORDER BY
            gi.date DESC
        LIMIT {n_games} -- Coger solo los partidos que nos interesan
    ),
    RankedGames AS (
        SELECT
            ps.player,
            ps.game_id,
            ps.date,
            {stats_columns},
            gi.season,
            ROW_NUMBER() OVER (
                PARTITION BY ps.player, gi.season
                ORDER BY gi.date DESC
            ) AS rank
        FROM
            player_stats ps
        JOIN
            game_info gi
            ON ps.game_id = gi.game_id
        JOIN
            AwayTeamPlayers atp
            ON ps.player = atp.player -- Asegura que solo los jugadores del equipo visitante se incluyen
        WHERE
            gi.season = {season}
            AND gi.date < (
                SELECT date FROM LastNGames
            )
    ),
    FilteredGames AS (
        SELECT
            rg.player,
            rg.game_id,
            rg.date,
            rg.rank,
            {stats_columns2}
        FROM
            RankedGames rg
        WHERE
            rg.rank <= {n_games}
    )
    SELECT
        fg.player,
        fg.game_id AS previous_game_id,
        fg.date AS previous_game_date,
        cg.current_game_id,
        cg.current_game_date,
        {stats_columns3}
    FROM
        FilteredGames fg
    JOIN
        CurrentGame cg
        ON fg.player IN (SELECT player from AwayTeamPlayers)
    ORDER BY
        fg.player, fg.rank;
      '''
  elif team == "home":
    roster_query = f'''
    WITH CurrentGame AS (
        SELECT
            gi.home_team,
            gi.game_id AS current_game_id,
            gi.date AS current_game_date
        FROM
            game_info gi
        WHERE
            gi.game_id = {game_id}
    ),
    AwayTeamPlayers AS (
        SELECT
            ps.player,
            ps.game_id
        FROM
            player_stats ps
        JOIN
            game_info gi
            ON ps.game_id = gi.game_id
        JOIN
            CurrentGame cg
            ON gi.game_id = cg.current_game_id
        WHERE
            ps.team = cg.home_team-- Solo jugadores del equipo local
    ),
    LastNGames AS (
        SELECT
            gi.date
        FROM
            game_info gi
        JOIN
            CurrentGame cg
            ON (gi.away_team = cg.home_team OR gi.home_team = cg.home_team)
        WHERE
            gi.date < cg.current_game_date -- Solo fechas anteriores
        ORDER BY
            gi.date DESC
        LIMIT {n_games} -- Coger solo los partidos que nos interesan
    ),
    RankedGames AS (
        SELECT
            ps.player,
            ps.game_id,
            ps.date,
            {stats_columns},
            gi.season,
            ROW_NUMBER() OVER (
                PARTITION BY ps.player, gi.season
                ORDER BY gi.date DESC
            ) AS rank
        FROM
            player_stats ps
        JOIN
            game_info gi
            ON ps.game_id = gi.game_id
        JOIN
            AwayTeamPlayers atp
            ON ps.player = atp.player -- Asegura que solo los jugadores del equipo visitante se incluyen
        WHERE
            gi.season = {season}
            AND gi.date < (
                SELECT date FROM LastNGames
            )
    ),
    FilteredGames AS (
        SELECT
            rg.player,
            rg.game_id,
            rg.date,
            rg.rank,
            {stats_columns2}
        FROM
            RankedGames rg
        WHERE
            rg.rank <= {n_games}
    )
    SELECT
        fg.player,
        fg.game_id AS previous_game_id,
        fg.date AS previous_game_date,
        cg.current_game_id,
        cg.current_game_date,
        {stats_columns3}
    FROM
        FilteredGames fg
    JOIN
        CurrentGame cg
        ON fg.player IN (SELECT player from AwayTeamPlayers)
    ORDER BY
        fg.player, fg.rank;
      '''
  return roster_query

In [ ]:
# Función para hacer las medianas de los últimos n partidos jugados por cada jugador del roster por cada partido

def get_full_stats (query, conn, team='away'):

  stats_df = pd.read_sql(query, con=conn)

  # Convertimos MP a float y además filtramos los jugadores que no jugaron por un motivo u otro
  stats_df['MP'] = stats_df['MP'].apply(lambda x: -99 if isinstance(x, str) and (x == 'Did Not Play' or x == 'Did Not Dress' or x == 'Not With Team' or x == 'Player Suspended') else float(x.split(':')[0]) + float(x.split(':')[1]) / 60 if isinstance(x, str) else x)
  stats_df = stats_df.drop(columns=['ORtg', 'DRtg', '+/-','GmSc','USG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%'])
  stats_df[['previous_game_id', 'current_game_id']] = stats_df[['previous_game_id', 'current_game_id']].astype(object)

  # Filtrar jugadores que jugaron y convertir 'MP' a tipo float
  stats_df_played = stats_df[stats_df['MP'] != '-99'].copy()
  stats_df_played.loc[:, 'MP'] = stats_df_played['MP'].astype(float)

  numeric_columns = stats_df_played.select_dtypes(include=['float64']).columns

  median_stats = stats_df_played.groupby('player')[numeric_columns].median()
  median_stats = median_stats[numeric_columns].round(3)

  cols_sum = ['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'BPM', 'PIE']
  cols_ratios = [{'3P%': ('3P', '3PA')}, {'3PAr': ('3PA', 'FGA')}, {'FTr': ('FTA', 'FGA')}, {'FT%': ('FT', 'FTA')}]
  # Comprobamos si hay datos en el Dataframe, puede que no haya datos para el partido si no se han alcanzado los n_games
  if stats_df.empty:
      print(f"No se encontraron datos para el partido. Saltando...")
      # Devuelve un DataFrame vacío
      return pd.DataFrame()

  # Comprobamos si las columnas de median_stats están en cols_sum
  missing_cols = [col for col in cols_sum if col not in median_stats.columns]
  # Si faltan columnas, las añadimos con valor 0
  for col in missing_cols:
      print(f"Falta la columna {col} en el partido {stats_df.current_game_id[0]}")

  # Sumar las columnas de median_stats
  suma_resultados = {col: median_stats[col].sum(skipna=True) for col in cols_sum}
  df_resultados = pd.DataFrame(suma_resultados, index=[0])

  # Calcular ratios
  for ratio_dict in cols_ratios:
      for key, (numerator, denominator) in ratio_dict.items():
          df_resultados[key] = round((df_resultados[numerator] / df_resultados[denominator]), 3)

  # Calcular eFG%
  df_resultados['eFG%'] = round(((df_resultados['FG'] + (0.5 * df_resultados['3P'])) / df_resultados['FGA']), 3)

  # Calcular TS%
  df_resultados['TS%'] = round((df_resultados['PTS'] / (2 * (df_resultados['FGA'] + 0.475 * df_resultados['FTA']))), 3)

  # Calcular TOV%
  df_resultados['TOV%'] = round((df_resultados['TOV'] / (df_resultados['FGA'] + 0.44 * df_resultados['FTA'] + df_resultados['TOV'])), 3)

  # Renombrar columnas según el equipo
  prefix = 'a_' if team == 'away' else 'h_'
  df_resultados.columns = [prefix + col for col in df_resultados.columns]

  return df_resultados

In [ ]:
# Versión inicial para obtener las estadísticas de los últimos n partidos jugados por cada jugador de cada equipo en cada partido de la temporada
# Se utilizó un bucle para guardar los csv por temporadas y después se concatenó por errores de ejecución en Google Colab dónde se perdía la información
# Se añade después una versión mejorada que no necesita guardar los csv por temporadas y conecta directamente con la base de datos
# Obtener las estadísticas de los últimos n partidos jugados por cada jugador de cada equipo en cada partido de la temporada
# Este código es "estático", ya que hay que cambiar el número de partidos y dataframe temporal para 10, 20 y 30 partidos

# seasons = game_info['season'].unique()

# for season in seasons:
#     print(f"Processing season: {season}")
#     season_games = game_info[game_info['season'] == season]['game_id'].unique()

#     # Inicializamos Dataframe
#     players_full_30_temp = pd.DataFrame()  

#     for game in season_games:
#         try:
#             print(f"Processing game: {game}")
#             away_team = game_info[game_info['game_id'] == game]['away_team'].values[0]
#             home_team = game_info[game_info['game_id'] == game]['home_team'].values[0]

#             # Obtener estadísticas de equipos
#             away_gid_query = roster_query(game_id=game, n_games=30, team='away')
#             away_stats = get_full_stats(query=away_gid_query, conn=conn, team='away')

#             home_gid_query = roster_query(game_id=game, n_games=30, team='home')
#             home_stats = get_full_stats(query=home_gid_query, conn=conn, team='home')

#             # Verificar si se obtuvieron estadísticas válidas
#             if away_stats.empty or home_stats.empty:
#                 print(f"No se encontraron datos para el partido {game}. Saltando...")
#                 continue

#             # Concatenar estadísticas de equipos
#             agg_stats = pd.concat([away_stats, home_stats], axis=1)
#             gid_info = game_info[game_info['game_id'] == game].reset_index(drop=True)

#             # Concatenar información del juego con estadísticas agregadas
#             stats = pd.concat([gid_info, agg_stats], axis=1)

#             # Agregar a players_full_XX_temp
#             players_full_30_temp = pd.concat([players_full_30_temp, stats], ignore_index=True)

#         except Exception as e:
#             print(f"Error processing game {game}: {e}")
#             continue  # Continuar con el siguiente juego

#     # Guardar DataFrame de la temporada como CSV
#     csv_filename = f"/content/drive/MyDrive/Data/players_full_30_{season}.csv"
#     players_full_30_temp.to_csv(csv_filename, index=False)
#     print(f"Saved data for season {season} to {csv_filename}")

Se han truncado las últimas 5000 líneas del flujo de salida.
Processing game: 232411220212
Processing game: 232411220213
Processing game: 232411220214
Processing game: 232411220215
Processing game: 232411220216
Processing game: 232411220217
Processing game: 232411220218
Processing game: 232411220219
Processing game: 232411220220
Processing game: 232411240221
Processing game: 232411240222
Processing game: 232411240223
Processing game: 232411240224
Processing game: 232411240225
Processing game: 232411240226
Processing game: 232411240227
Processing game: 232411240228
Processing game: 232411240229
Processing game: 232411240230
Processing game: 232411250231
Processing game: 232411250232
Processing game: 232411250233
Processing game: 232411250234
Processing game: 232411250235
Processing game: 232411250236
Processing game: 232411260237
Processing game: 232411260238
Processing game: 232411260239
Processing game: 232411260240
Processing game: 232411260241
Processing game: 232411260242
Processin

In [ ]:
# Versión inicial para obtener las estadísticas de los últimos n partidos jugados por cada jugador de cada equipo en cada partido de la temporada
# Se utilizó un bucle para guardar los csv por temporadas y después se concatenó por errores de ejecución en Google Colab dónde se perdía la información
# Se añade después una versión mejorada que no necesita guardar los csv por temporadas y conecta directamente con la base de datos

# data_path = "/content/drive/MyDrive/Data/"

# players_full_10 = pd.DataFrame()
# players_full_30 = pd.DataFrame()

# seasons = [910, 1011, 1112, 1213, 1314, 1415, 1516, 1617, 1718, 1819, 1920, 2021, 2122, 2223, 2324]
# games = [10, 30]

# for game in games:
#   for season in seasons:
#     df = pd.read_csv(data_path + f"players_full_{game}_{season}.csv")
#     if game == 10:
#       players_full_10 = pd.concat([players_full_10, df])
#     else:
#       players_full_30 = pd.concat([players_full_30, df])

# players_full_10.to_csv(data_path + "players_full_10.csv", index=False)
# players_full_30.to_csv(data_path + "players_full_30.csv", index=False)


In [ ]:
# Versión definitiva para obtener las estadísticas de los últimos n partidos jugados por cada jugador de cada equipo en cada partido de la temporada
# Tiempo de ejecución alto, no se recomienda ejecutar en Google Colab por posible pérdida de conexión

# Función para verificar si una tabla existe en la base de datos
def table_exists(conn, table_name):
    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'"
    result = pd.read_sql(query, conn)
    return not result.empty

# Procesamos cada temporada
seasons = game_info['season'].unique()

for season in seasons:
    print(f"Processing season: {season}")
    season_games = game_info[game_info['season'] == season]['game_id'].unique()
    
    # Inicializamos DataFrame para almacenar datos de la temporada
    players_full_30_temp = pd.DataFrame()  

    for game in season_games:
        try:
            print(f"Processing game: {game}")
            away_team = game_info[game_info['game_id'] == game]['away_team'].values[0]
            home_team = game_info[game_info['game_id'] == game]['home_team'].values[0]

            # Obtener estadísticas de equipos
            away_gid_query = roster_query(game_id=game, n_games=30, team='away')
            away_stats = get_full_stats(query=away_gid_query, conn=conn, team='away')

            home_gid_query = roster_query(game_id=game, n_games=30, team='home')
            home_stats = get_full_stats(query=home_gid_query, conn=conn, team='home')

            # Verificar si se obtuvieron estadísticas válidas
            if away_stats.empty or home_stats.empty:
                print(f"No se encontraron datos para el partido {game}. Saltando...")
                continue

            # Concatenar estadísticas de equipos
            agg_stats = pd.concat([away_stats, home_stats], axis=1)
            gid_info = game_info[game_info['game_id'] == game].reset_index(drop=True)

            # Concatenar información del juego con estadísticas agregadas
            stats = pd.concat([gid_info, agg_stats], axis=1)

            # Agregar a players_full_30_temp
            players_full_30_temp = pd.concat([players_full_30_temp, stats], ignore_index=True)

        except Exception as e:
            print(f"Error processing game {game}: {e}")
            continue  # Continuar con el siguiente juego

    # Si no hay datos para la temporada, saltamos a la siguiente
    if players_full_30_temp.empty:
        print(f"No se han procesado datos para la temporada {season}.")
        continue

    # Nombre de la tabla en SQLite
    table_name = "player_full_30"
    
    # Verificamos si la tabla ya existe
    if not table_exists(conn, table_name):
        # La tabla no existe, la creamos e insertamos todos los datos
        players_full_30_temp.to_sql(table_name, conn, index=False)
        print(f"Tabla {table_name} creada e insertados todos los datos de la temporada {season}.")
    else:
        # La tabla existe, consultamos los game_id que ya están en la base de datos
        query_existing = f"SELECT DISTINCT game_id FROM {table_name}"
        existing_game_ids = pd.read_sql(query_existing, conn)['game_id'].tolist()
        
        # Filtramos para obtener solo las filas cuyos game_id no estén en la tabla
        new_rows = players_full_30_temp[~players_full_30_temp['game_id'].isin(existing_game_ids)]
        
        if not new_rows.empty:
            # Insertamos las filas nuevas
            new_rows.to_sql(table_name, conn, index=False, if_exists='append')
            print(f"Datos actualizados para la temporada {season}. Nuevos partidos insertados: {len(new_rows)}")
        else:
            print(f"No hay nuevos datos para actualizar en la temporada {season}.")

# Finalmente, guardar los cambios y cerrar la conexión si ya no se necesita más
conn.commit()
conn.close()